In [114]:
# pip install ucimlrepo
import numpy as np
import pandas as pd

In [116]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
car_evaluation = fetch_ucirepo(id=19) 
  
# data (as pandas dataframes) 
X = car_evaluation.data.features 
y = car_evaluation.data.targets 
  
# metadata 
print(car_evaluation.metadata) 
  
# variable information 
print(car_evaluation.variables)

{'uci_id': 19, 'name': 'Car Evaluation', 'repository_url': 'https://archive.ics.uci.edu/dataset/19/car+evaluation', 'data_url': 'https://archive.ics.uci.edu/static/public/19/data.csv', 'abstract': 'Derived from simple hierarchical decision model, this database may be useful for testing constructive induction and structure discovery methods.', 'area': 'Other', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 1728, 'num_features': 6, 'feature_types': ['Categorical'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1988, 'last_updated': 'Thu Aug 10 2023', 'dataset_doi': '10.24432/C5JP48', 'creators': ['Marko Bohanec'], 'intro_paper': {'ID': 249, 'type': 'NATIVE', 'title': 'Knowledge acquisition and explanation for multi-attribute decision making', 'authors': 'M. Bohanec, V. Rajkovič', 'venue': '8th Intl Workshop on Expert Systems and their Applications, 

In [118]:
# print(X)
# print(y)

X_encoded = pd.get_dummies(X)
y_encoded = pd.get_dummies(y)

print(X_encoded)
print(y_encoded)

      buying_high  buying_low  buying_med  buying_vhigh  maint_high  \
0           False       False       False          True       False   
1           False       False       False          True       False   
2           False       False       False          True       False   
3           False       False       False          True       False   
4           False       False       False          True       False   
...           ...         ...         ...           ...         ...   
1723        False        True       False         False       False   
1724        False        True       False         False       False   
1725        False        True       False         False       False   
1726        False        True       False         False       False   
1727        False        True       False         False       False   

      maint_low  maint_med  maint_vhigh  doors_2  doors_3  ...  doors_5more  \
0         False      False         True     True    False  ...      

In [99]:
from sklearn.model_selection import train_test_split

# data split, 70% training and 30% temp (temp = validation + test)
X_train, X_temp, y_train, y_temp = train_test_split(X_encoded, y_encoded, test_size = 0.3, random_state = 42)

# 30% temp data into 15% validation and 15% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 42)

In [100]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()

In [101]:
classifier.fit(X_train, y_train)

DecisionTreeClassifier()

In [108]:
from sklearn.metrics import accuracy_score, classification_report

# Make predictions on the validation set
y_val_pred = classifier.predict(X_val)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_val_pred)
print(f"Validation Accuracy: {accuracy:.2f}")

# Print classification report for more detailed metrics
print("Classification Report:")
print(classification_report(y_val, y_val_pred))

Validation Accuracy: 0.95
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.88      0.91        57
           1       0.73      0.80      0.76        10
           2       0.98      0.99      0.99       178
           3       0.80      0.86      0.83        14

   micro avg       0.95      0.95      0.95       259
   macro avg       0.86      0.88      0.87       259
weighted avg       0.95      0.95      0.95       259
 samples avg       0.95      0.95      0.95       259



In [112]:
from sklearn.metrics import accuracy_score, classification_report

# Make predictions on the test set
y_test_pred = classifier.predict(X_test)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy:.2f}")

# Print classification report for more detailed metrics
print("Classification Report on Test Data:")
print(classification_report(y_test, y_test_pred))

Test Accuracy: 0.95
Classification Report on Test Data:
              precision    recall  f1-score   support

           0       0.93      0.85      0.89        61
           1       0.69      1.00      0.82         9
           2       0.97      0.99      0.98       180
           3       0.88      0.70      0.78        10

   micro avg       0.95      0.95      0.95       260
   macro avg       0.87      0.89      0.87       260
weighted avg       0.95      0.95      0.95       260
 samples avg       0.95      0.95      0.95       260

